# **Grad CAM(Gradient Class Activation Map)**
* CAM(Class Activation Map): CNN의 특정 클래스에 대한 활성화 지도를 시각화하는 기법으로, 해당 클래스와 연관된 이미지 영역을 강조하여 모델이 어떤 부분에 주목
* Grad-CAM (Gradient-weighted Class Activation Map): 마지막 convolution layer의 특성과 해당 클래스의 그래디언트를 결합해 클래스별로 중요한 이미지 영역을 시각
* 과정 
    1. Forward Propagation: 이미지를 모델에 입력하여, 마지막 convolution layer의 feature map과 출력 예측값(softmax 결과) 
    2. Backpropagation: 관심 있는 클래스에 대해 예측값의 그래디언트를 계산하여, 마지막 convolution layer의 각 채널(feature map)에 대한 기여도 계산
    3. Weight 계산: 각 채널의 중요도를 나타내는 가중치(α_k)를 그래디언트를 평균내어 계산
    4. Weighted Combination: 가중치와 feature map을 결합하여 클래스에 대한 활성화 맵을 계산
    5. Upsampling: 활성화 맵을 입력 이미지 크기로 보간(upsampling)하여 관심 있는 영역을 시각화

<img src="Contents_XAI/Grad-CAM.png" alt="Grad-CAM" width="800" height="400"/>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import resnet50,ResNet50
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

2024-12-16 08:04:40.089710: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-16 08:04:40.116272: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-16 08:04:40.510642: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


: 

In [ ]:
# 사전 학습된 신경망 모델을 불러오고 신경망 구조를 확인
#model=ResNet50(weights='imagenet')
model = ResNet50(weights=None)
model.load_weights('./resnet50_weights.h5') 
model.summary()

# 지정된 영상을 불러와 크기 조정하고 화면에 디스플레이
image_path='./hummingbird.jpg'
img=image.load_img(image_path,target_size=(224,224))
plt.matshow(img)

# 영상을 신경망 입력 형태로 변환
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
x=resnet50.preprocess_input(x)

# 인식을 시도하고 top-5 결과를 출력
preds=model.predict(x)
print("예측 결과:", resnet50.decode_predictions(preds,top=5)[0])

In [ ]:
# 신경망 모델의 특징 추출(컨볼루션층) 부분에서 마지막 층을 지정
# 특징 추출 부분만으로 구성된 모델 model_1 만들기
last_conv_layer=model.get_layer("conv5_block3_out")

model_1=keras.Model(model.inputs,last_conv_layer.output)

In [ ]:
# 분류(전역평균 풀링 또는 완전연결층) 부분만으로 구성된 모델 model_2 만들기
input_2=keras.Input(shape=last_conv_layer.output.shape[1:])
x_2=model.get_layer("avg_pool")(input_2)
x_2=model.get_layer("predictions")(x_2) 
model_2=keras.Model(input_2,x_2)

In [ ]:
# GradientTape 함수를 이용한 그레이디언트 계산
with tf.GradientTape() as tape:
    output_1=model_1(x)
    tape.watch(output_1) # 마지막 층으로 미분하기 위한 준비
    preds=model_2(output_1)
    class_id=tf.argmax(preds[0])
    output_2=preds[:,class_id]

grads=tape.gradient(output_2,output_1) # 그레이디언트 계산
pooled_grads=tf.reduce_mean(grads,axis=(0,1,2)) 

output_1=output_1.numpy()[0]
pooled_grads=pooled_grads.numpy()
for i in range(pooled_grads.shape[-1]): 
    output_1[:,:,i]*=pooled_grads[i]
heatmap=np.mean(output_1,axis=-1)

heatmap=np.maximum(heatmap,0)/np.max(heatmap) # [0,1]로 정규화
plt.matshow(heatmap)

In [1]:
# 열지도를 입력 영상에 덧씌움
img=image.load_img(image_path) # 입력 영상을 다시 받아옴

img=image.img_to_array(img)
heatmap=np.uint8(255*heatmap) # [0,255]로 변환

jet=cm.get_cmap("jet") # [0,255] 열지도를 jet 컬러맵으로 표시함
color=jet(np.arange(256))[:,:3]
color_heatmap=color[heatmap]

color_heatmap=keras.preprocessing.image.array_to_img(color_heatmap)
color_heatmap=color_heatmap.resize((img.shape[1],img.shape[0]))
color_heatmap=keras.preprocessing.image.img_to_array(color_heatmap)

overlay_img=color_heatmap*0.4+img # 덧씌움
overlay_img=keras.preprocessing.image.array_to_img(overlay_img)
plt.matshow(overlay_img)

NameError: name 'image' is not defined